In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/dataset"

# role = "arn:aws:iam::235411143540:role/SageMakerRole"# claude_perrin_3
# role = "arn:aws:iam::146258905784:role/SageMakerRole"# claude_perrin_5
role = "arn:aws:iam::224769526787:role/SageMakerRole"# claude_perrin_9

print(bucket)

In [ ]:
# inputs = sagemaker_session.upload_data(path="saved_dataset", bucket=bucket, key_prefix=prefix)
# print("input spec (in this case, just an S3 path): {}".format(inputs))
# Train set and Test set
# common_bucket_name = "s3://sagemaker-eu-central-1-235411143540"
# train = (f"{common_bucket_name}/train_directory")
# test = (f"{common_bucket_name}/test_dir" )

common_bucket_name = f"s3://{bucket}"

# train = (f"{common_bucket_name}/crop_train_set")
# test = (f"{common_bucket_name}/crop_test_set")
train = (f"{common_bucket_name}/train_set")
train = (f"{common_bucket_name}/cropped_train")
test = (f"{common_bucket_name}/cropped_train")
print(train,'\n',test)

In [ ]:
!pygmentize code/faster_rcnn.py


In [ ]:
from sagemaker.pytorch import PyTorch
instance_count = 2
if instance_count > 1:
    distribution={"torch_distributed": {"enabled": True}} #torch_distributed
else:
    distribution = None

estimator = PyTorch(
    entry_point="train.py",
    source_dir='.',        # Directory where your source code is located
    role=role,
    py_version="py310",
    framework_version="2.0.1",
    instance_count=instance_count,
    # distribution=distribution,
    instance_type="ml.m4.xlarge", # ml.g4dn.xlarge ml.m4.xlarge	
    hyperparameters={"epochs": 2, "backend": "gloo"},
)


In [ ]:
estimator.fit({"train": train, "test": test})


In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")


In [ ]:
import gzip
import numpy as np
import random
import os

data_dir = "data/MNIST/raw"
with gzip.open(os.path.join(data_dir, "t10k-images-idx3-ubyte.gz"), "rb") as f:
    images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28).astype(np.float32)

mask = random.sample(range(len(images)), 16)  # randomly select some of the test images
mask = np.array(mask, dtype=np.int32)
data = images[mask]


In [ ]:
response = predictor.predict(np.expand_dims(data, axis=1))
print("Raw prediction result:")
print(response)
print()

labeled_predictions = list(zip(range(10), response[0]))
print("Labeled predictions: ")
print(labeled_predictions)
print()

labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print("Most likely answer: {}".format(labeled_predictions[0]))


In [ ]:
sagemaker_session.delete_endpoint(endpoint_name=predictor.endpoint_name)
